# DETERMINE R function of $\theta$

- I dati nel .txt sono separati dal carattere '\t  ', disposti in 2 colonne relativi al tempo (in millisecondi) e alla funzione di correlazione

- $\sigma_{g^2}$ = 0.001 l'incertezza è fissata all'ultima ciffra significativa

- i dati validi da tenere in considerazione sono quelli nell'intervallo 0.1ms-10ms

- Gli angoli nel nome del file sono l'angolo supplementare di $\theta$ (per l'errore di misurazione)

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import os
import sys
import plotly.graph_objects as go 
dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')
from labbiofisica import Interpolazione, final_val

nel caricamento dei dati nelle strutture numpy i millisecondi vengono convertiti in secondi

In [2]:
# filename = f'./data2/g_{θ}_2'
# read = pd.read_table(filename, skiprows=30, nrows=245 - 30)
# # read.head()

# # print(read[read.columns[4]])
# print(read.loc[:, read.columns[2:11]].std(axis=1))

In [3]:
# Carico i dati {θ: {t: [], g: [], sigmag: []}}

data = {}

TETA = [40,50,60,70,80,90,100,110,120,130] 
GAMMA = []
sigmaGAMMA = []

# t convertito in secondi, si considerano dati solo tra 0.1 e 10 ms a causa del rumore
for θ in TETA:
    filename = f'./data2/g_{θ}_2.asc'
    
    read = pd.read_csv(filename, skiprows=30, nrows=245 - 30, sep='\t', encoding='ISO-8859-1')

    # print(read.columns[2:12])

    read = read[(read['Lag [ms]'] >= 0.1) & (read['Lag [ms]'] <= 10)]
    read['std'] = read.loc[:, read.columns[2:12]].std(axis=1) #  / np.sqrt(10 - 1)
    
    t = read['Lag [ms]'].to_numpy()/1000
    g = read['Average'].to_numpy()
    σg = read['std'].to_numpy()

    # θ = 180 - θ
    data[θ] = {'t':t,'g':g,'sigmag': σg} # converto in secondi

# TETA = 180 - TETA # inverto angoli supplementari
TETA = np.array(TETA)
# print(data)

In [23]:
# Constants
n = 1.33 # index of refraction
# sigman = 0.01 # uncertainty of the index of refraction
η = 10**-9  # g/(nm*s) viscosity of water
# sigmaη = 0.01 * 10**-9  # uncertainty of the viscosity of water
λ = 632.8  # nm wavelength of the laser
T = 293  # K temperature
sigmaT = 1  # K uncertainty of the temperature
KB = 1.3806e-2  # g (nm)^2 / s^2 K boltzmann constant

# INTERPOLAZIONE

attraverso al funzione: $g^2(\tau) = A e^{-\gamma \tau} + B$

In [5]:
# fit data
CHI_r = []
DOF = []
PVALUE = []

f = lambda x, A, B, γ: A*np.exp(-γ*x) + B

for key in data.keys():
    x = data[key]['t']
    y = data[key]['g']
    sigmay = data[key]['sigmag']
    m = Interpolazione(x, y, sigmay, f, [0.05, 0.001, 2.7],['A','B','γ'])
    data[key]['fit'] = m
    # print(m.values['γ'])
    data[key]['gamma'] = m.values['γ']
    GAMMA.append(m.values['γ'])
    sigmaGAMMA.append(m.errors['γ'])
    CHI_r.append(m.rchi2)
    DOF = m.dof
    PVALUE.append(m.pvalue)

GAMMA = np.array(GAMMA) 
sigmaGAMMA = np.array(sigmaGAMMA)


In [6]:
# solo per stampare altri valori legati ai fit
AsA = {}
for θ in data.keys():
    A = data[θ]['fit'].values['A']
    sigmaA = data[θ]['fit'].errors['A']
    B = data[θ]['fit'].values['B']
    sigmaB = data[θ]['fit'].errors['B']
    AsA[θ] = {'A':A,'σA':sigmaA,'B':B,'σB':sigmaB}

Asa = pd.DataFrame(AsA)
display(Asa)

,40,50,60,70,80,90,100,110,120,130
A,0.078358,0.073376,0.076864,0.072878,0.077385,0.073208,0.073449,0.073629,0.068494,0.068619
σA,0.000497,0.000316,0.000445,0.000501,0.000680,0.000909,0.000992,0.001067,0.001231,0.001323
B,0.001716,0.000603,0.000226,0.000233,0.000106,0.000204,0.000074,0.000020,0.000078,0.000102
σB,0.000307,0.000141,0.000133,0.000125,0.000103,0.000103,0.000085,0.000088,0.000088,0.000084


In [7]:
# print FIT RESULT:
GAMMAFVAL = [final_val(g,sg,2,3,'Hz') for g,sg in zip(GAMMA,sigmaGAMMA)]
gammateta = pd.DataFrame({'θ':TETA,'γ':GAMMA,'σγ':sigmaGAMMA,'γ_out':GAMMAFVAL,'pvalue':PVALUE,'chir':CHI_r,'dof':DOF})

print('---------------------------FIT RESULT----------------------------')
display(gammateta)

# print(gammateta['γ'].to_numpy())
# print(gammateta['σγ'].to_numpy())

---------------------------FIT RESULT----------------------------


,θ,γ,σγ,γ_out,pvalue,chir,dof
0,40,527.056266,9.484879,(0.53 ± 0.01)e3 Hz,1.0,0.17,50
1,50,829.623716,7.026010,(0.83 ± 0.01)e3 Hz,1.0,0.34,50
2,60,1197.095239,11.120392,(1.2 ± 0.01)e3 Hz,1.0,0.17,50
3,70,1621.361270,18.255080,(1.62 ± 0.02)e3 Hz,1.0,0.17,50
4,80,2008.897986,25.656633,(2.01 ± 0.03)e3 Hz,1.0,0.16,50
5,90,2448.485713,37.552854,(2.45 ± 0.04)e3 Hz,1.0,0.09,50
6,100,2792.436701,46.492427,(2.79 ± 0.05)e3 Hz,1.0,0.16,50
7,110,3274.390615,57.720813,(3.27 ± 0.06)e3 Hz,1.0,0.16,50
8,120,3660.831500,66.826641,(3.66 ± 0.07)e3 Hz,1.0,0.17,50
9,130,4009.246478,84.094518,(4.01 ± 0.08)e3 Hz,1.0,0.13,50


# PLOT FIT

In [8]:
colors = px.colors.sequential.Turbo

fig = go.Figure()
for i,θ in enumerate(data.keys()):
    x = data[θ]['t']
    y = data[θ]['g']
    γ = data[θ]['gamma']
    sigmay = data[θ]['sigmag']
    X,Y = data[θ]['fit'].draw()

    # plt.plot(X,Y)
    # plt.xlim(0,1)
    # plt.show()

    fig.add_trace(go.Scatter(x=X, y=Y, mode='lines',line_color=colors[i], showlegend=False,hoverinfo='skip'))

    fig.add_trace(go.Scatter(x=x,y=y, mode='markers', name=f'θ={θ}°',
        marker=dict(color = colors[i]),
        error_y=dict( 
			type='data', 
			array=sigmay, 
			color=colors[i],
			thickness=1.5, 
			width=3, 
		)))

fig.update_layout(
        xaxis_type="log",
        yaxis_title="g<sup>2</sup>(τ)",
        xaxis_title="t (s)",
        # title='g<sup>2</sup>(τ) vs t',
        xaxis=dict(range=[np.log10(0.0001), np.log10(0.010)]),
        width=800,
        height=600,
        # title={
        #     'text': "Correlation Function g<sup>2</sup>(τ)",
        #     'y':0.9,
        #     'x':0.5,
        #     'xanchor': 'center',
        #     'yanchor': 'top',
        #     'font':dict(size=30)
        # },
        plot_bgcolor='white',
        font=dict(
            #family="Courier New, monospace",
            size=18,
            color="Black"
        ),
            # yaxis=dict(range=[0, 0.06])
    )

fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)

fig.show()

$Q = \frac{4\pi n}{\lambda}\sin\frac{\theta}{2}$

$\gamma = 2DQ^2 \Rightarrow D = \frac{\gamma\lambda^2}{32\pi^2n^2\sin^2\frac{\theta}{2}}$

$D = \frac{K_B T}{6\pi\eta\R}$

$R = \frac{16\pi n^2 K_B T}{3\eta\lambda^2\gamma}\sin^2\frac{\theta}{2}$

n = 1.33

$\eta$ = 0.01poise

$\lambda$ = 633nm

$K_B$ = 1.3806 $\cdot 10^{-2} g(nm)^2s^{-2}K^{-1}$

T = 293K

In [15]:
# fit gamma teta

def gamma_2DQ2(θ,D):
    theta = np.deg2rad(θ)
    return 2*(n**2)*D*16*(np.pi**2)*np.sin(theta/2)**2/(λ**2)

gammatetafit = Interpolazione(TETA, GAMMA, sigmaGAMMA, gamma_2DQ2, [3000000],['D'])
print(gammatetafit)

D,σD = gammatetafit.values['D'],gammatetafit.errors['D']

#D = n2D/(n**2)
#σD = D*np.sqrt((σn2D/n2D)**2 + (sigman/n)**2)

# print('n2D = coefficiente diffusione * n^2 = ',final_val(n2D,σn2D,2,exp=6,udm='nm^2/s'))
print('D = coefficiente diffusione = ',final_val(D,σD,2,exp=6,udm='m^2/s'))

----------------- VALORI FIT: -----------------
D: (3.426 ± 0.014)e6 

dof: 9
chi2: 38.26
chi2 ridotto: 4.25
pvalue: 0.0
------------------------------------------------

D = coefficiente diffusione =  (3.43 ± 0.01)e6 m^2/s


In [22]:
# plot fit gamma teta

fig = go.Figure()

X,Y = gammatetafit.draw()
fig.add_trace(go.Scatter(x=X, y=Y, mode='lines',line_color=px.colors.sequential.Turbo[3], showlegend=False,hoverinfo='skip'))

fig.add_trace(go.Scatter(x=TETA,y=GAMMA, mode='markers',showlegend=False,
    marker=dict(color=px.colors.sequential.Turbo[1]),
    error_y=dict( 
        type='data', 
        array=sigmaGAMMA, 
        color=px.colors.sequential.Turbo[1],
        thickness=1.5, 
        width=3, 
    )))

fig.update_layout(
    # xaxis_type="log",
    yaxis_title="γ (Hz)",
    xaxis_title="θ (°)",
    # title='g<sup>2</sup>(τ) vs t',
    # xaxis=dict(range=[np.log10(0.0001), np.log10(0.010)]),
    width=600,
    height=400,
    # title={
    #     'text': "γ = 2DQ<sup>2</sup>(θ)",
    #     'y':0.8,
    #     'x':0.5,
    #     'xanchor': 'center',
    #     'yanchor': 'top',
    #     'font':dict(size=15)
    # },
    plot_bgcolor='white',
    font=dict(
        #family="Courier New, monospace",
        size=10,
        color="Black"
    ),
        # yaxis=dict(range=[0, 0.06])
)

fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
# fig.update_layout(font=dict(size=10))
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)

fig.show()

# R CON COEFF DI DIFFUSIONE:

In [24]:
R = KB*T/(6*np.pi*η*D)
σR = R*np.sqrt((sigmaT/T)**2 + (σD/D)**2)

print('R = ',final_val(R,σR,1,udm='nm'))

R =  62.6 ± 0.3 nm


# TEST COMPATIBILITA'

In [25]:
from scipy.stats import norm

costruttoreD = 120
sigmaDcostruttore = 15

D = R*2
σD = 2*σR

σ = np.sqrt(σD**2 + sigmaDcostruttore**2)

t = (D-costruttoreD)/σ


print('Diametro misurato: ',final_val(D,σD,1,0,udm='nm'))
print('Diametro costruttore: ',final_val(costruttoreD,sigmaDcostruttore,0,0,udm='nm'))
print(f"t = {np.round(t,3)}")
print(f"COmpatibilità: {np.round(norm.cdf(t)*100,1)}%")

# NOTA: il t non deve essere maggiore di 2, la probabilità deve essere minore possibile

Diametro misurato:  125.3 ± 0.7 nm
Diametro costruttore:  120 ± 15 nm
t = 0.353
COmpatibilità: 63.8%
